In [14]:
import pandas as pd
import numpy as np
import string
import os
import sys
import re
from collections import Counter, defaultdict
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import nltk
from rank_bm25 import BM25Okapi
import itertools
import time
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [2]:
qrels = pd.read_csv("D:\Courses\Sem 7 2021-22\COL764\COL764 Project\FIRE2017-IRLeD-track-data\Task_2\irled-qrel.txt", sep = ' ', header=None)

In [3]:
# is this stopword list the correct one ? can we try other stopword collections or may be custom ?
stopword_set = set(stopwords.words('english'))
def stopword_removal(word_list):

    return [word for word in word_list if word not in stopword_set ]    


# do we make the case lower for legal documents?
def tokenize_document(doc, lowercase = False, removeStopwords = True, stemming = False, stemmer = 'PorterStemmer',
                      tokenizer = 'RegExpTokeinzer', lemmatize = True):
    
    # punctuations are removed with empty string
    doc = doc.translate(str.maketrans('', '', string.punctuation))
    
    # do we convert to lowercase?
    if(lowercase):
        doc = doc.lower()
    
    
    # variety of tokenizers could be experimented with
    if(tokenizer == 'RegExpTokeinzer'):
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(doc)
    
    # word tokenizer of nltk - a bit slow since based on NLP techniques
    elif(tokenizer == 'Word_tokenizer'):
        tokens = word_tokenize(doc)
    
        
    if(removeStopwords == True):
        tokens = stopword_removal(tokens)
    
        
    #variety of stemmers could be experimented with
    if(stemming):
        if(stemmer == 'PorterStemmer'):
            tokens = [ps.stem(token) for token in tokens ]
        
    
    # We can also perform lemmatization if we want
    # write function for lemmatizer here.
    if(lemmatize == True):
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    
    
    return tokens
        


In [17]:
def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.
 
    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.
 
    Args:
        doc: string to be converted
 
    Returns:
        list of synsets
 
    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    tokens = nltk.word_tokenize(doc)
    pos = nltk.pos_tag(tokens)
    tags = [tag[1] for tag in pos]
    wntag = [convert_tag(tag) for tag in tags]
    ans = list(zip(tokens,wntag))
    sets = [wn.synsets(x,y) for x,y in ans]
    final = [val[0] for val in sets if len(val) > 0]
     
    return final

def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
     
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None

In [18]:
# this function is for processing the corpus
def process_corpus(filename, lowercase = False, removeStopwords = True, stemming = False, stemmer = 'PorterStemmer',
                    tokenizer = 'RegExpTokeinzer'):
    corpus_files = sorted(os.listdir(filename))
    
    list_sysnets = []
    
    indexes_of_doc = defaultdict(lambda:0)
    
    count = 0
    for corpus_file in corpus_files:
        filepath = os.path.join(filename, corpus_file)
        
        indexes_of_doc[count] = corpus_file[:-4]
        print(corpus_file)
        with open(filepath, 'r') as f:
            doc_text = f.readlines()
            
            
            doc_text = ''.join(doc_text)
            
            # this considers all words
            tokens = tokenize_document(doc_text, lowercase = lowercase, removeStopwords = removeStopwords , stemming = stemming, 
                                       stemmer = stemmer , tokenizer =  tokenizer)
            
            pos = nltk.pos_tag(tokens)
            tags = [tag[1] for tag in pos]
            wntag = [convert_tag(tag) for tag in tags]
            ans = list(zip(tokens,wntag))
            sets = [wordnet.synsets(x,y) for x,y in ans]
            final = [val[0] for val in sets if len(val) > 0]
            
            list_sysnets.append(final)
        
        count +=1
    
    return list_sysnets, indexes_of_doc
            

            


In [19]:
list_tokens, indexes_of_doc = process_corpus("D:\Courses\Sem 7 2021-22\COL764\COL764 Project\FIRE2017-IRLeD-track-data\Task_2\Prior_Cases", lowercase = True)

prior_case_0001.txt
prior_case_0002.txt
prior_case_0003.txt
prior_case_0004.txt
prior_case_0005.txt
prior_case_0006.txt
prior_case_0007.txt
prior_case_0008.txt
prior_case_0009.txt
prior_case_0010.txt
prior_case_0011.txt
prior_case_0012.txt
prior_case_0013.txt
prior_case_0014.txt
prior_case_0015.txt
prior_case_0016.txt
prior_case_0017.txt
prior_case_0018.txt
prior_case_0019.txt
prior_case_0020.txt
prior_case_0021.txt
prior_case_0022.txt
prior_case_0023.txt
prior_case_0024.txt
prior_case_0025.txt
prior_case_0026.txt
prior_case_0027.txt
prior_case_0028.txt
prior_case_0029.txt
prior_case_0030.txt
prior_case_0031.txt
prior_case_0032.txt
prior_case_0033.txt
prior_case_0034.txt
prior_case_0035.txt
prior_case_0036.txt
prior_case_0037.txt
prior_case_0038.txt
prior_case_0039.txt
prior_case_0040.txt
prior_case_0041.txt
prior_case_0042.txt
prior_case_0043.txt
prior_case_0044.txt
prior_case_0045.txt
prior_case_0046.txt
prior_case_0047.txt
prior_case_0048.txt
prior_case_0049.txt
prior_case_0050.txt


prior_case_0412.txt
prior_case_0413.txt
prior_case_0414.txt
prior_case_0415.txt
prior_case_0416.txt
prior_case_0417.txt
prior_case_0418.txt
prior_case_0419.txt
prior_case_0420.txt
prior_case_0421.txt
prior_case_0422.txt
prior_case_0423.txt
prior_case_0424.txt
prior_case_0425.txt
prior_case_0426.txt
prior_case_0427.txt
prior_case_0428.txt
prior_case_0429.txt
prior_case_0430.txt
prior_case_0431.txt
prior_case_0432.txt
prior_case_0433.txt
prior_case_0434.txt
prior_case_0435.txt
prior_case_0436.txt
prior_case_0437.txt
prior_case_0438.txt
prior_case_0439.txt
prior_case_0440.txt
prior_case_0441.txt
prior_case_0442.txt
prior_case_0443.txt
prior_case_0444.txt
prior_case_0445.txt
prior_case_0446.txt
prior_case_0447.txt
prior_case_0448.txt
prior_case_0449.txt
prior_case_0450.txt
prior_case_0451.txt
prior_case_0452.txt
prior_case_0453.txt
prior_case_0454.txt
prior_case_0455.txt
prior_case_0456.txt
prior_case_0457.txt
prior_case_0458.txt
prior_case_0459.txt
prior_case_0460.txt
prior_case_0461.txt


prior_case_0823.txt
prior_case_0824.txt
prior_case_0825.txt
prior_case_0826.txt
prior_case_0827.txt
prior_case_0828.txt
prior_case_0829.txt
prior_case_0830.txt
prior_case_0831.txt
prior_case_0832.txt
prior_case_0833.txt
prior_case_0834.txt
prior_case_0835.txt
prior_case_0836.txt
prior_case_0837.txt
prior_case_0838.txt
prior_case_0839.txt
prior_case_0840.txt
prior_case_0841.txt
prior_case_0842.txt
prior_case_0843.txt
prior_case_0844.txt
prior_case_0845.txt
prior_case_0846.txt
prior_case_0847.txt
prior_case_0848.txt
prior_case_0849.txt
prior_case_0850.txt
prior_case_0851.txt
prior_case_0852.txt
prior_case_0853.txt
prior_case_0854.txt
prior_case_0855.txt
prior_case_0856.txt
prior_case_0857.txt
prior_case_0858.txt
prior_case_0859.txt
prior_case_0860.txt
prior_case_0861.txt
prior_case_0862.txt
prior_case_0863.txt
prior_case_0864.txt
prior_case_0865.txt
prior_case_0866.txt
prior_case_0867.txt
prior_case_0868.txt
prior_case_0869.txt
prior_case_0870.txt
prior_case_0871.txt
prior_case_0872.txt


prior_case_1233.txt
prior_case_1234.txt
prior_case_1235.txt
prior_case_1236.txt
prior_case_1237.txt
prior_case_1238.txt
prior_case_1239.txt
prior_case_1240.txt
prior_case_1241.txt
prior_case_1242.txt
prior_case_1243.txt
prior_case_1244.txt
prior_case_1245.txt
prior_case_1246.txt
prior_case_1247.txt
prior_case_1248.txt
prior_case_1249.txt
prior_case_1250.txt
prior_case_1251.txt
prior_case_1252.txt
prior_case_1253.txt
prior_case_1254.txt
prior_case_1255.txt
prior_case_1256.txt
prior_case_1257.txt
prior_case_1258.txt
prior_case_1259.txt
prior_case_1260.txt
prior_case_1261.txt
prior_case_1262.txt
prior_case_1263.txt
prior_case_1264.txt
prior_case_1265.txt
prior_case_1266.txt
prior_case_1267.txt
prior_case_1268.txt
prior_case_1269.txt
prior_case_1270.txt
prior_case_1271.txt
prior_case_1272.txt
prior_case_1273.txt
prior_case_1274.txt
prior_case_1275.txt
prior_case_1276.txt
prior_case_1277.txt
prior_case_1278.txt
prior_case_1279.txt
prior_case_1280.txt
prior_case_1281.txt
prior_case_1282.txt


prior_case_1644.txt
prior_case_1645.txt
prior_case_1646.txt
prior_case_1647.txt
prior_case_1648.txt
prior_case_1649.txt
prior_case_1650.txt
prior_case_1651.txt
prior_case_1652.txt
prior_case_1653.txt
prior_case_1654.txt
prior_case_1655.txt
prior_case_1656.txt
prior_case_1657.txt
prior_case_1658.txt
prior_case_1659.txt
prior_case_1660.txt
prior_case_1661.txt
prior_case_1662.txt
prior_case_1663.txt
prior_case_1664.txt
prior_case_1665.txt
prior_case_1666.txt
prior_case_1667.txt
prior_case_1668.txt
prior_case_1669.txt
prior_case_1670.txt
prior_case_1671.txt
prior_case_1672.txt
prior_case_1673.txt
prior_case_1674.txt
prior_case_1675.txt
prior_case_1676.txt
prior_case_1677.txt
prior_case_1678.txt
prior_case_1679.txt
prior_case_1680.txt
prior_case_1681.txt
prior_case_1682.txt
prior_case_1683.txt
prior_case_1684.txt
prior_case_1685.txt
prior_case_1686.txt
prior_case_1687.txt
prior_case_1688.txt
prior_case_1689.txt
prior_case_1690.txt
prior_case_1691.txt
prior_case_1692.txt
prior_case_1693.txt


In [ ]:
list_tokens[0]

In [7]:
def findall(p, s):

    i = s.find(p)
    while i != -1:
        yield i
        
        i = s.find(p, i+1)
        

def process_query(doc):
    citation_marker = "[?CITATION?]"
    
    return findall(citation_marker, doc)

In [8]:
# function to get the query text by selecting region around the marker, how much to take?

# feedback: why construct the whole list ls1 and ls2, rather find one by one upto 40 spaces on either sides.
# i guess it would be more efficient.
def process_markers(s, i, p):
    
    ls1 = [(a.start(), a.end()) for a in list(re.finditer(' ', s[:i]))]
    
    # another hyper-parameter,take max 40 spaces
    index1 = min(40, len(ls1))
    

    

    # find spaces
    ls2 = [(a.start(), a.end()) for a in list(re.finditer(' ', s[i + len(p):]))]
    
    # another hyper-parameter,take max 40 spaces
    index2 = min(39, len(ls2)-1)
    
    
    # only considering spaces for now
    # may be later add logic for full stops and \n chars.
    return s[:i][ls1[-index1][0]:] + s[i+ len(p):][0: ls2[index2][0]]


def process_queries(filename):
    query_files = os.listdir(filename)
    
    case_queries = defaultdict(lambda:[])
    
    for query_file in query_files:
        
        filepath = os.path.join(filename, query_file)
        
        
        
        with open(filepath, 'r') as f:
            doc_text = f.readlines() 
            doc_text = ''.join(doc_text)
            
        citation_marker_indices = list(process_query(doc_text))
        
        print(query_file, len(citation_marker_indices))
        
        for index in citation_marker_indices:
            query_text = process_markers(doc_text, index, "[?CITATION?]")
            
            case_queries[query_file[:-4]].append(query_text)
            
    return case_queries


In [9]:
# remove the pattern "27\."
queries = process_queries("D:\Courses\Sem 7 2021-22\COL764\COL764 Project\FIRE2017-IRLeD-track-data\Task_2\Current_Cases")

current_case_0001.txt 8
current_case_0002.txt 15
current_case_0003.txt 19
current_case_0004.txt 32
current_case_0005.txt 15
current_case_0006.txt 13
current_case_0007.txt 26
current_case_0008.txt 27
current_case_0009.txt 13
current_case_0010.txt 37
current_case_0011.txt 11
current_case_0012.txt 20
current_case_0013.txt 30
current_case_0014.txt 37
current_case_0015.txt 10
current_case_0016.txt 10
current_case_0017.txt 77
current_case_0018.txt 16
current_case_0019.txt 19
current_case_0020.txt 10
current_case_0021.txt 13
current_case_0022.txt 29
current_case_0023.txt 28
current_case_0024.txt 13
current_case_0025.txt 14
current_case_0026.txt 17
current_case_0027.txt 5
current_case_0028.txt 23
current_case_0029.txt 22
current_case_0030.txt 8
current_case_0031.txt 22
current_case_0032.txt 25
current_case_0033.txt 12
current_case_0034.txt 8
current_case_0035.txt 10
current_case_0036.txt 7
current_case_0037.txt 37
current_case_0038.txt 15
current_case_0039.txt 26
current_case_0040.txt 15
curre

In [33]:
# chodrow similarity

def similarity_score(s1, s2):
    s =[]
    for i1 in s1:
        #print(i1)
        r = []
        scores = [x for x in [i1.path_similarity(i2) for i2 in s2] if x is not None]
        if scores:
            s.append(max(scores))
    return sum(s)/len(s)


def similarity(syn1, syn2, metric = 'lch'):
    if(metric == 'lch'):
        a = str(syn1)
        #print(str(syn1), str(syn2))
        if(str(syn1) == '[]' or str(syn2) == '[]'):
            return 0
        elif (re.search(r"\.(.*?)\.", str(syn1)).group(1) == re.search(r"\.(.*?)\.", str(syn2)).group(1)):
            sim = syn1.lch_similarity(syn2)
            if sim == None:
                return 0
            else:
                return sim
        
        return 0
    
def calculate_similarity(query_net, metric=''):
    doc_scores = []
    for i in range(len(list_tokens)):
        time1 = time.time()
        doc = list_tokens[i]
#         sim_list = []
#         for sysnet in query_net:
#             sysnet_sim = 0
#             for token_sysnet in doc:
#                 sysnet_sim = max(sysnet_sim, similarity(sysnet, token_sysnet[0]) if len(token_sysnet)  > 0 else 0)
#             if(sysnet_sim > 0):
#                 sim_list.append(sysnet_sim)
#         doc_scores.append(np.mean(sim_list))
        
        score = similarity_score(query_net, doc)
        
        doc_scores.append(score)
        
        time2 = time.time()
        print(doc_scores[i], time2-time1)
    
    return doc_scores
                
def sysnet_similarity(queryname, lowercase = False, removeStopwords = True, stemming = False, stemmer = 'PorterStemmer',
               tokenizer = 'RegExpTokeinzer'):
    
    query_scores_doc = defaultdict(lambda:0)
    
    for query in queries[queryname]:
        
        # using all terms 
        tokens = tokenize_document(query, lowercase = lowercase, removeStopwords = removeStopwords, stemming = stemming, 
                                         stemmer = stemmer, tokenizer = tokenizer)
        
        pos = nltk.pos_tag(tokens)
        tags = [tag[1] for tag in pos]
        wntag = [convert_tag(tag) for tag in tags]
        ans = list(zip(tokens,wntag))
        sets = [wordnet.synsets(x,y) for x,y in ans]
        final = [val[0] for val in sets if len(val) > 0]
        
        query_sysnet = final
        
        #query_sysnet = [wordnet.synsets(token)[0] if len(wordnet.synsets(token)) > 0 else [] for token in query_tokens]
        
        
        doc_scores = calculate_similarity(query_sysnet, metric='')
        
        print(doc_scores)
        
        indices = np.argsort(doc_scores)[::-1]
        
        values = [doc_scores[i] for i in indices]
        

        for i in range(len(indices)):
            index = indices[i]
            score = values[i]
            
            query_scores_doc[index] = query_scores_doc[index]  if query_scores_doc[index] > score else score

     
    results = sorted(query_scores_doc.items(), key = lambda x: x[1], reverse = True)
    
    
    rankings = []
    
    for result in results:
        index = result[0]
        rankings.append(indexes_of_doc[index])
        
        
    
    return rankings


In [34]:
sysnet_similarity("current_case_0001", lowercase = True)

0.6201680672268908 3.205005407333374
0.9819819819819818 84.30360150337219
0.7809523809523808 6.8221471309661865
0.6995098039215686 5.377956867218018
0.7524193548387097 6.926005601882935
0.6386072708653354 3.2525458335876465
0.8104938271604937 4.0549635887146
0.8502886002886002 10.441287279129028
0.808080808080808 7.007036447525024
0.565079365079365 1.1270020008087158
0.7872549019607843 11.402623414993286
0.3983614951356886 0.7000401020050049
0.6664285714285714 1.8359649181365967
0.7358585858585858 4.449995517730713
0.7197916666666666 2.1380460262298584
0.6132034632034632 2.0429906845092773
0.9247619047619046 18.708594799041748
0.4272273425499231 0.40296244621276855
0.8782828282828283 22.58186149597168
0.6852150537634408 3.0650038719177246
0.9342105263157895 24.892018795013428
0.782106782106782 2.8441593647003174
0.8363636363636363 6.348501443862915
0.704040404040404 5.0702502727508545
0.8048340548340548 6.026983976364136
0.633410138248848 1.7433879375457764
0.7204301075268816 6.3162164

0.9861111111111112 30.613469123840332
0.8888888888888888 10.05199933052063
0.8342857142857143 8.6625394821167
0.7126262626262625 3.5410549640655518
0.23530423280423282 0.13299775123596191
0.657258064516129 2.532858371734619
0.8904761904761904 9.52399230003357


ZeroDivisionError: division by zero

In [113]:
syn1 = wordnet.synset('here.r.02')
syn2 = wordnet.synset('herein.r.01')

# Synset('inside.r.02') Synset('herein.r.01')

In [112]:
print(syn1.lch_similarity(syn2))

WordNetError: Computing the lch similarity requires Synset('hera.n.01') and Synset('herein.r.01') to have the same part of speech.

In [70]:
syn2.v

AttributeError: 'Synset' object has no attribute 'v'

In [77]:

a = str(syn1)
print(a)
m = re.search(r"\.(.*?)\.", a)
print (m.group(1))

Synset('hello.n.01')
n


In [28]:
similarity_score(list_tokens[0], list_tokens[1])

Synset('air.n.01')
Synset('scandium.n.01')
Synset('six.n.01')
Synset('seven.n.01')
Synset('five.n.01')
Synset('scale.n.01')
Synset('eleven.n.01')
Synset('october.n.01')


KeyboardInterrupt: 